# Пояснения к практике по деревьям решений

Допустим, у нас есть следующее множество:

X1 | X2| Y
---|---|--
 1 | 3 | 0
 2 | 4 | 0
 3 | 4 | 1
 2 | 2 | 0
 1 | 4 | 1

Это матрица, в которой собрана информация по пяти объектам. Каждый объект имеет два измерения-признака: **X1** и **X2**, и отнесён к одному из двух классов - 0 или 1 (столбец **Y**).

При построении деревьев решений это множество рекурсивно разбивается на подмножества до тех пор, пока в каждом листе не окажутся примеры только одного класса. В нашем случае - либо все нули, либо все единицы.

Необходимо выбрать признак, по которому будем разбивать множество. У нас есть выбор - можем взять либо признак **X1**, либо **X2**. Чтобы сделать математически обоснованный выбор, воспользуемся энтропийным критерием. Нам нужно взять такой признак, при котором снимется наибольшая неопределённость (энтропия). А для этого нам нужно посчитать энтропию исходного множества.

## Расчёт энтропии множества

Формула энтропии:

$Info(T) = -\sum_{i=1}^Np_{i}log_{2}(p_{i})$

В этой формуле **T** - это исходное множество. В нашем случае это столбец **Y**, то есть значения `[0, 0, 1, 0, 1]`. **N** - это количество классов. У нас их два - ноль и единица. Количество классов можно найти одним из способов:

In [25]:
Y = np.array([0, 0, 1, 0, 1])
N = len(np.unique(Y))  # 1 способ
N = len(set(Y))        # 2 способ
N

2

Кроме того, нам нужно знать вероятность встретить каждый из классов в множестве. Для этого нужно просто поделить количество элементов данного класса на размер множества. Например, вероятность для класса "0" будет равна в данном примере $\frac{3}{5}$, так как в **Y** ноль встречается 3 раза, а всего элементов 5. Вероятность для класса "1" будет равна $\frac{2}{5}$. Вот пример кода:

In [30]:
# мы можем выполнить Y == item потому, что Y - это np.array, а не просто list
p = [sum(Y == item) / len(Y) for item in set(Y)]
p

[0.6, 0.4]

Рассмотрим этот код детально. Для начала цикл. Он просто перебирает уникальные значения из **Y**. item на первой итерации  равен нулю, затем единице:

In [53]:
[item for item in set(Y)]

[0, 1]

Размер множества можно вычислить через `len(Y)`:

In [54]:
len(Y)

5

Посчитать количество элементов с данным значением (например, с нулём) можно так:

In [55]:
sum(Y == 0)

3

Вероятность встретить нулевое значение:

In [56]:
sum(Y == 0) / len(Y)

0.6

Осталось только посчитать итоговое значение энтропии:

In [33]:
info = -sum([p[i] * np.log2(p[i]) for i in range(len(p))])
info

0.9709505944546686

Можно обернуть всё это в функцию:

In [34]:
def Info(T):
    p = [sum(T == item) / len(T) for item in set(T)]
    return -sum([p[i] * np.log2(p[i]) for i in range(len(p))])

In [35]:
Info(Y)

0.9709505944546686

А если посчитать энтропию для множества, состоящего из одинаковых значений, то мы ожидаемо получим 0, ведь никакой неопределённости в таком множестве нет:

In [39]:
zeros = np.array([0, 0, 0, 0, 0])
Info(zeros) == 0.0

True

# Расчёт энтропии после разбиения

Допустим, мы решили разбить наше исходное множество по признаку **X1**. Этот признак содержит значения `[1, 2, 3, 2, 1]`. Из них уникальные значения `[1, 2, 3]`. Таким образом, мы можем разбить исходное множество на 3 подмножества по признаку **X1**:

Было:

X1 | X2| Y
---|---|--
 **1** | 3 | 0
 **2** | 4 | 0
 **3** | 4 | 1
 **2** | 2 | 0
 **1** | 4 | 1

Стало:

X1 | X2| Y
---|---|--
 **1** | 3 | 0
 **1** | 4 | 1
 
 X1 | X2| Y
---|---|--
 **2** | 4 | 0
 **2** | 2 | 0
 
 X1 | X2| Y
---|---|--
 **3** | 4 | 1

Для каждого из подмножеств мы можем вычислить энтропию. Используем для этого последний столбец - метку класса:

In [40]:
dataset = np.array([
    [1, 3, 0],
    [2, 4, 0],
    [3, 4, 1],
    [2, 2, 0],
    [1, 4, 1]])

In [42]:
X1_values = dataset[:, 0]
X1_values

array([1, 2, 3, 2, 1])

In [57]:
Y = dataset[:, -1]
Y

array([0, 0, 1, 0, 1])

In [49]:
for item in np.unique(X1_values):
    print('Уникальное значение из столбца X1:', item)
    
    print('Подмножество, соответствующие значению %d из X1:\n' % item, dataset[X1_values == item, :])
    
    T = Y[X1_values == item]
    print('Значения Y, соответствующие значению %d из X1:' % item, T)
    
    print('Соответствующая энтропия:', Info(T))
    
    print()

Уникальное значение из столбца X1: 1
Подмножество, соответствующие значению 1 из X1:
 [[1 3 0]
 [1 4 1]]
Значения Y, соответствующие значению 1 из X1: [0 1]
Соответствующая энтропия: 1.0

Уникальное значение из столбца X1: 2
Подмножество, соответствующие значению 2 из X1:
 [[2 4 0]
 [2 2 0]]
Значения Y, соответствующие значению 2 из X1: [0 0]
Соответствующая энтропия: -0.0

Уникальное значение из столбца X1: 3
Подмножество, соответствующие значению 3 из X1:
 [[3 4 1]]
Значения Y, соответствующие значению 3 из X1: [1]
Соответствующая энтропия: -0.0



Кроме того, значения энтропии для подмножеств мы должны взвесить весовыми коэффициентами, которые рассчитываются просто как отношение размера подмножества к размеру исходного множества ($\frac{2}{5}=0.4$, $\frac{2}{5}=0.4$ и $\frac{1}{5}=0.2$ соответственно):

In [50]:
coefs = [sum(X1_values == item) / len(X1_values) for item in np.unique(X1_values)]
coefs

[0.4, 0.4, 0.2]

Соберём всё вместе и посчитаем энтропию после разбиения на подмножества по признаку **X1**:

In [51]:
info_s = 0

for item in np.unique(X1_values):
    coef = sum(X1_values == item) / len(X1_values)
    info_s += coef * Info(Y[X1_values == item])

info_s

0.4

И итоговый информационный выигрыш:

In [52]:
gain = Info(Y) - info_s
gain

0.5709505944546686

Можно обернуть всё в функцию:

In [58]:
def gain(dataset, feat_index):
    
    Y = dataset[:, -1]
    X_values = dataset[:, feat_index]
    
    info_s = 0
    for item in np.unique(X_values):
        coef = sum(X_values == item) / len(X_values)
        info_s += coef * Info(Y[X_values == item])
    
    return Info(Y) - info_s

In [59]:
gain(dataset, 0)  # выигрыш по признаку 1

0.5709505944546686

In [60]:
gain(dataset, 1)  # выигрыш по признаку 2

0.4199730940219749

Выигрыш от разбиения по первому признаку оказался выше, поэтому логично выбрать именно его.